# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/sajith/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sajith/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sajith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("select * from InsertTableName", engine)

In [4]:
df.isna().any()

id                         True
message                   False
original                   True
genre                     False
related                    True
request                   False
offer                     False
aid_related               False
medical_help              False
medical_products          False
search_and_rescue         False
security                  False
military                  False
child_alone               False
water                     False
food                      False
shelter                   False
clothing                  False
money                     False
missing_people            False
refugees                  False
death                     False
other_aid                 False
infrastructure_related    False
transport                 False
buildings                 False
electricity               False
tools                     False
hospitals                 False
shops                     False
aid_centers               False
other_in

In [5]:
df = df.dropna(subset=['related'], axis=0)

In [6]:
df.isna().any()

id                         True
message                   False
original                   True
genre                     False
related                   False
request                   False
offer                     False
aid_related               False
medical_help              False
medical_products          False
search_and_rescue         False
security                  False
military                  False
child_alone               False
water                     False
food                      False
shelter                   False
clothing                  False
money                     False
missing_people            False
refugees                  False
death                     False
other_aid                 False
infrastructure_related    False
transport                 False
buildings                 False
electricity               False
tools                     False
hospitals                 False
shops                     False
aid_centers               False
other_in

In [7]:
# X = df.iloc[:,2]
# Y = df.iloc[:,4:]
df['child_alone'].value_counts(dropna =False)

0    26028
Name: child_alone, dtype: int64

In [8]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
X.isna().sum()

0

In [9]:
Y.isna().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [10]:
Y['request'] = Y['request'].astype('int')

In [11]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1.0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [12]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [13]:
def tokenize(text):
    # remove URL
    text = re.sub(url_regex, " ", text.lower())
    # Normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokernize test
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not  in stop_words ]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
def build_pipeline(estimator):
    pipeline = Pipeline([
        ('transformer', Pipeline([
            ('vect', CountVectorizer(tokenizer = tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('clf', estimator)
     ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 0)

In [ ]:
tokenize(X_test.iloc[0])

In [ ]:
estimators = []
estimator1 = OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',
                                          alpha=1e-4,  random_state=42,
                                          shuffle=True, n_jobs=5))
estimators.append(estimator1)
estimator2 = OneVsRestClassifier(MultinomialNB())
estimators.append(estimator2)
estimator3 = OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=5)
estimators.append(estimator3)
estimator4 = OneVsRestClassifier(RandomForestClassifier(n_jobs = 5))
estimators.append(estimator4)

In [17]:
multi_label_estimators = []
multi_label_estimator1 = MultiOutputClassifier(OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',
                                          alpha=1e-4,  random_state=42,
                                          shuffle=True, n_jobs=5)))
multi_label_estimators.append(multi_label_estimator1)

multi_label_estimator2 = MultiOutputClassifier(OneVsRestClassifier(MultinomialNB()))
multi_label_estimators.append(multi_label_estimator2)

multi_label_estimator3 = MultiOutputClassifier(OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=5))
multi_label_estimators.append(multi_label_estimator3)

multi_label_estimator4 = MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier(n_jobs = 5)))
multi_label_estimators.append(multi_label_estimator4)

In [ ]:
multi_label_estimator1.get_params().keys()

In [19]:
parameters = { 
    'transformer__vect__max_features': [100],
    'transformer__vect__ngram_range': ((1,1),(1,2)),
    'transformer__tfidf__use_idf': (True, False),
    'clf__n_estimators': [50, 100, 200],
    'clf__min_samples_split': [2, 3, 4]
}

for i , estimator in enumerate(multi_label_estimators):
    pipeline = build_pipeline(estimator)
    pipeline.fit(X_train, Y_train)
    prediction = pipeline.predict(X_test)
    print(classification_report(Y_test, prediction, target_names=Y.columns.values))
    

/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 0.0 is present in all training examples.
  str(classes[c]))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

                        precision    recall  f1-score   support

               related       0.86      0.93      0.89      3983
               request       0.80      0.58      0.68       931
                 offer       0.00      0.00      0.00        20
           aid_related       0.77      0.68      0.72      2226
          medical_help       0.66      0.21      0.32       447
      medical_products       0.68      0.27      0.39       284
     search_and_rescue       0.74      0.14      0.23       170
              security       0.00      0.00      0.00       104
              military       0.66      0.21      0.32       189
           child_alone       0.00      0.00      0.00         0
                 water       0.83      0.62      0.71       371
                  food       0.81      0.73      0.76       597
               shelter       0.79      0.58      0.67       477
              clothing       0.74      0.38      0.50        77
                 money       0.59      

/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 0.0 is present in all training examples.
  str(classes[c]))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

                        precision    recall  f1-score   support

               related       0.78      0.99      0.87      3983
               request       0.92      0.23      0.37       931
                 offer       0.00      0.00      0.00        20
           aid_related       0.75      0.61      0.68      2226
          medical_help       0.50      0.00      0.00       447
      medical_products       0.00      0.00      0.00       284
     search_and_rescue       0.00      0.00      0.00       170
              security       0.00      0.00      0.00       104
              military       0.00      0.00      0.00       189
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       371
                  food       0.75      0.03      0.05       597
               shelter       0.50      0.00      0.00       477
              clothing       0.00      0.00      0.00        77
                 money       0.00      

/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being 

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      3983
               request       0.83      0.53      0.64       931
                 offer       0.00      0.00      0.00        20
           aid_related       0.79      0.66      0.72      2226
          medical_help       0.68      0.14      0.23       447
      medical_products       0.66      0.17      0.26       284
     search_and_rescue       1.00      0.06      0.12       170
              security       0.00      0.00      0.00       104
              military       0.62      0.08      0.14       189
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.50      0.62       371
                  food       0.84      0.60      0.70       597
               shelter       0.82      0.46      0.59       477
              clothing       0.71      0.16      0.26        77
                 money       0.75      

/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 0.0 is present in all training examples.
  str(classes[c]))


                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      3983
               request       0.86      0.52      0.65       931
                 offer       0.00      0.00      0.00        20
           aid_related       0.77      0.69      0.73      2226
          medical_help       0.63      0.06      0.11       447
      medical_products       0.71      0.08      0.15       284
     search_and_rescue       0.56      0.03      0.06       170
              security       1.00      0.01      0.02       104
              military       0.92      0.06      0.11       189
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.39      0.54       371
                  food       0.81      0.66      0.73       597
               shelter       0.81      0.40      0.54       477
              clothing       0.80      0.10      0.18        77
                 money       0.75      

/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sajith/Documents/Acedamic/Projects/Analytics/DRPipeline/disaster-response-pipeline/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being 

In [ ]:
for i , estimator in enumerate(estimators):
    pipeline = build_pipeline(estimator)
    pipeline.fit(X_train, Y_train['child_alone'])
    prediction = pipeline.predict(X_test[:2])
    print(prediction)
    print("Score {}".format(cv.score(X_test, Y_test)))

In [ ]:
parameters = { 
    'transformer__vect__ngram_range': ((1, 1), (1, 2)),
    'transformer__vect__max_df': (0.5, 0.75, 1.0),
    'transformer__tfidf__use_idf': (True, False)
}
cv = GridSearchCV(pipeline, param_grid = parameters)

In [ ]:
#cv.fit(X_train,Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = { 
    'transformer__vect__max_features': [100],
    'transformer__vect__ngram_range': ((1,1),(1,2)),
    'transformer__tfidf__use_idf': (True, False),
    'clf__n_estimators': [50, 100, 200],
    'clf__min_samples_split': [2, 3, 4]
}

cv = GridSearchCV(pipeline2, param_grid = parameters)

In [ ]:
cv.fit(X_train,Y_train)

In [ ]:
cv.score(X_test, Y_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.